In [ ]:
! pip install -qU \
    openai \
    pinecone-client \
    langchain \
    tiktoken \
    dotenv

In [135]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

## Initializing Embedding Model and Vector DB

In [132]:
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings


model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name
)

In [136]:
PINECONE_API = config['PINECONE_API']
PINECONE_ENV = config['PINECONE_ENV']

In [137]:
import pinecone

# find API key in console at app.pinecone.io
YOUR_API_KEY = PINECONE_API
# find ENV (cloud region) next to API key in console
YOUR_ENV = PINECONE_ENV

index_name = 'llm-recommender-system'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

In [5]:
index = pinecone.Index(index_name)
index_stats_response = index.describe_index_stats()

print(index_stats_response)

{'dimension': 1536,
 'index_fullness': 0.001,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}


## Indexing Embedding Vectors

In [8]:
import pandas as pd

In [9]:
sample_books = pd.read_csv("../data/sample_books.csv")

In [10]:
sample_books

,Id,Name,Description
0,1374307,"Tough Times Never Last, but Tough People Do!","Name your problem, and you name your possibili..."
1,1334737,Black Gold,This bracing novel introduces vivid African-Am...
2,1598771,The Autobiography of Lincoln Steffens,Rediscover a man Americans turned to not only ...
3,2179832,Bright Shiny Morning,One of the most celebrated and controversial a...
4,4421731,The Socially Skilled Child Molester: Different...,Know what signs indicate a child molester!<br ...
...,...,...,...
95,2442839,"National Portrait Gallery, London",-- The first full catalogue of this major art ...
96,2787050,Collecting Toy Cars and Trucks: A Collector's ...,This indispensable identification and value gu...
97,4506874,Self Related Cognitions In Anxiety And Motivation,First published in 1986. Routledge is an impri...
98,3002398,Predators: A Pop-up Book with Revolutionary Te...,Get up close and personal with some of the wor...


In [11]:
sample_books['category'] =  sample_books.apply(lambda x: 'popular' if x.name < 50 else 'recommended', axis=1)

In [12]:
sample_books['user_id'] = sample_books.apply(lambda x: x.name % 10 + 1 if x['category'] == 'recommended' else None, axis = 1)

In [13]:
sample_books['ranking'] = sample_books.groupby('user_id').cumcount() + 1

In [14]:
sample_books.rename(columns={'Id' : 'id', 'Name': 'title', 'Description': 'description'}, inplace=True)

In [94]:
data = sample_books

In [96]:
data[data['user_id'] == 1]

,id,title,description,category,user_id,ranking
50,4589735,Informal Economy: A Research G,First published in 1991. Routledge is an impri...,recommended,1.0,1.0
60,1112628,In Amma's Healing Room: Gender and Vernacular ...,"""[I]t is extremely salubrious to see the ways ...",recommended,1.0,2.0
70,3041182,Vision & Prophecy in Amos Revised,In 1955 John D. W. Watts presented the faculty...,recommended,1.0,3.0
80,2243440,Environmental Regulation: Statutory Supplement...,This excellent resource takes full advantage o...,recommended,1.0,4.0
90,4502407,Hooded Americanism: The History of the Ku Klux...,"""The only work that treats Ku Kluxism for the ...",recommended,1.0,5.0


In [105]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [
    {
        'category': record['category'],
        'title': record['title'],
        'description': record['description'],
        **({'user_id': str(int(record['user_id']))} if record['category'] == 'recommended' else {})
    }
    for _, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['description']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['id'].astype(str)
    # add everything to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


In [106]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.001,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}

## Creating Vector Store and Querying

In [6]:
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.callbacks.manager import (
        AsyncCallbackManagerForRetrieverRun,
        CallbackManagerForRetrieverRun,
    )
from typing import (
    TYPE_CHECKING,
    Any,
    Callable,
    ClassVar,
    Collection,
    Dict,
    Iterable,
    List,
    Optional,
    Tuple,
    Type,
    TypeVar,
)
from langchain.schema.document import Document

class PineconeVectorStoreRetriever(VectorStoreRetriever):

     def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        if self.search_type == "similarity":
            docs_and_similarities = (
                self.vectorstore.similarity_search_with_relevance_scores(
                    query, **self.search_kwargs
                )
            )
            for doc, score in docs_and_similarities:
                doc.metadata = {**doc.metadata, **{"score": 1-score}}
            docs = [doc for doc, _ in docs_and_similarities]
        elif self.search_type == "similarity_score_threshold":
            docs_and_similarities = (
                self.vectorstore.similarity_search_with_relevance_scores(
                    query, **self.search_kwargs
                )
            )
            for doc, score in docs_and_similarities:
                doc.metadata = {**doc.metadata, **{"score": 1-score}}
            docs = [doc for doc, _ in docs_and_similarities]
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs 

In [7]:
from langchain.vectorstores import Pinecone

class PineconeModified(Pinecone):
    
    def as_retriever(self, **kwargs: Any) -> VectorStoreRetriever:
        """Return VectorStoreRetriever initialized from this VectorStore.

        Args:
            search_type (Optional[str]): Defines the type of search that
                the Retriever should perform.
                Can be "similarity" (default), "mmr", or
                "similarity_score_threshold".
            search_kwargs (Optional[Dict]): Keyword arguments to pass to the
                search function. Can include things like:
                    k: Amount of documents to return (Default: 4)
                    score_threshold: Minimum relevance threshold
                        for similarity_score_threshold
                    fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)
                    lambda_mult: Diversity of results returned by MMR;
                        1 for minimum diversity and 0 for maximum. (Default: 0.5)
                    filter: Filter by document metadata

        Returns:
            VectorStoreRetriever: Retriever class for VectorStore.

        Examples:

        .. code-block:: python

            # Retrieve more documents with higher diversity
            # Useful if your dataset has many similar documents
            docsearch.as_retriever(
                search_type="mmr",
                search_kwargs={'k': 6, 'lambda_mult': 0.25}
            )

            # Fetch more documents for the MMR algorithm to consider
            # But only return the top 5
            docsearch.as_retriever(
                search_type="mmr",
                search_kwargs={'k': 5, 'fetch_k': 50}
            )

            # Only retrieve documents that have a relevance score
            # Above a certain threshold
            docsearch.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8}
            )

            # Only get the single most similar document from the dataset
            docsearch.as_retriever(search_kwargs={'k': 1})

            # Use a filter to only retrieve documents from a specific paper
            docsearch.as_retriever(
                search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
            )
        """
        tags = kwargs.pop("tags", None) or []
        tags.extend(self._get_retriever_tags())

        return PineconeVectorStoreRetriever(vectorstore=self, **kwargs, tags=tags)

In [8]:
text_field = "description"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = PineconeModified(
    index, embed.embed_query, text_field
)
     

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [9]:
query = "adventure story"

vectorstore.similarity_search_with_score(
    query, 
    k=3,
    filter={'category': 'popular'}
)
     

[(Document(page_content="Hope and Have introduces us to a young orphan who undergoes a miraculous transformation. Ther first part of the story tells how Fanny, through trials, triumphs, and her faith in God, has a change of heart. The second part of the story follows Fanny's adventures during the terrible Indian massacre of 1862. The gripping story of how Fanny and a young man named Ethan French escape makes this a captivating book.", metadata={'category': 'popular', 'title': 'Hope and Have'}),
  0.81618315),
 (Document(page_content="Combining a phonics-based text with a funny story, the books in this new series are aimed at beginner readers. The pictures give clues to the words and fold out flaps on some pages give children the chance to guess what will happen next'", metadata={'category': 'popular', 'title': 'Fat Cat on a Mat'}),
  0.811931491),
 (Document(page_content="This bracing novel introduces vivid African-American characters into the traditional territory of James Michener an

In [10]:
query = "history"

vectorstore.similarity_search_with_score(
    query, 
    k=3,
    filter={'user_id' : '1' , 'category': 'recommended'}
)
     

[(Document(page_content='"The only work that treats Ku Kluxism for the entire period of it\'s existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin', metadata={'category': 'recommended', 'title': 'Hooded Americanism: The History of the Ku Klux Klan, 3rd ed.', 'user_id': '1'}),
  0.781868577),
 (Document(page_content="This excellent resource takes full advantage of on-line information and students' familiarity with it. Reflecting the author's dual focus on environmental regulations and the policy considerations that shape them.", metadata={'category': 'recommended', 'title': 'Environmental Regulation: Statutory Supplement an

## Tools definition


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.schema.vectorstore import VectorStore
from langchain.chains.query_constructor.ir import StructuredQuery, Visitor
from langchain.schema.language_model import BaseLanguageModel
from langchain.retrievers.self_query.pinecone import PineconeTranslator
from langchain.chains.query_constructor.base import load_query_constructor_runnable

from typing import Any, Dict, List, Optional, Sequence, Tuple, Type, Union

In [12]:
def _get_builtin_translator(vectorstore: VectorStore) -> Visitor:
    """Get the translator class corresponding to the vector store class."""
    BUILTIN_TRANSLATORS: Dict[Type[VectorStore], Type[Visitor]] = {
        PineconeModified: PineconeTranslator,
    }
    
    if vectorstore.__class__ in BUILTIN_TRANSLATORS:
        return BUILTIN_TRANSLATORS[vectorstore.__class__]()
    else:
        raise ValueError(
            f"Self query retriever with Vector Store type {vectorstore.__class__}"
            f" not supported."
        )

class PineconeSelfQueryRetriever(SelfQueryRetriever):

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        vectorstore: VectorStore,
        document_contents: str,
        metadata_field_info: Sequence[Union[AttributeInfo, dict]],
        structured_query_translator: Optional[Visitor] = None,
        chain_kwargs: Optional[Dict] = None,
        enable_limit: bool = False,
        use_original_query: bool = False,
        **kwargs: Any,
    ) -> "SelfQueryRetriever":
        if structured_query_translator is None:
            structured_query_translator = _get_builtin_translator(vectorstore)
        chain_kwargs = chain_kwargs or {}

        if "allowed_comparators" not in chain_kwargs:
            chain_kwargs[
                "allowed_comparators"
            ] = structured_query_translator.allowed_comparators
        if "allowed_operators" not in chain_kwargs:
            chain_kwargs[
                "allowed_operators"
            ] = structured_query_translator.allowed_operators
        query_constructor = load_query_constructor_runnable(
            llm,
            document_contents,
            metadata_field_info,
            enable_limit=enable_limit,
            **chain_kwargs,
        )
        return cls(
            query_constructor=query_constructor,
            vectorstore=vectorstore,
            use_original_query=use_original_query,
            structured_query_translator=structured_query_translator,
            **kwargs,
        )

In [13]:
# chat completion llm

llm = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.0
)

In [15]:
# conversational memory

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

### Tool 1 - Generic Recommendation

In [16]:
metadata_field_info=[
    AttributeInfo(
        name="user_id",
        description="The user ID of the book recommendation",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="The type of entry (popular or recommended)",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="title",
        description="The title of the book",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="The description of the book",
        type="float"
    ),
]
document_content_description = "The description of the book"

pineconeSelfQueryRetriever = PineconeSelfQueryRetriever.from_llm(llm, vectorstore, document_content_description, metadata_field_info, verbose=True)

In [18]:
## USER ID IS SPECIFICALLY DETERMINED IN FRONTEND

In [19]:
# retrieval qa chain

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pineconeSelfQueryRetriever,
    return_source_documents = True
)

In [22]:
qa({"query": "I am user 1 and recommend some books for me"})

{'query': 'I am user 1 and recommend some books for me',
 'result': 'As an AI, I don\'t have information about your specific interests. However, based on the context provided, you might be interested in the following books:\n\n1. "Hooded Americanism" - This book provides a comprehensive history of the Ku Klux Klan.\n2. "Vision and Prophecy in Amos" by John D. W. Watts - This is a classic in biblical prophecy studies.\n3. A book on environmental regulations and policy considerations - The title isn\'t provided in the context, but it\'s described as an excellent resource that takes full advantage of online information. \n\nPlease provide more information about your interests for more personalized recommendations.',
 'source_documents': [Document(page_content='First published in 1991. Routledge is an imprint of Taylor &amp; Francis, an informa company.', metadata={'category': 'recommended', 'title': 'Informal Economy: A Research G', 'user_id': '1'}),
  Document(page_content="This excellen

### Tool 2 - Popular Recommendation

In [91]:
# retrieval qa chain

popular_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'category': 'popular'}}),
    return_source_documents = True
)

### Tool 3 - Specific Recommendation

In [90]:
# retrieval qa chain

recommended_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'user_id' : '1' , 'category': 'recommended'}}),
    return_source_documents = True
)

In [125]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from operator import itemgetter

# Check if the recommended books is match the user's query
chain = (
    {"recommended_books": recommended_qa, "query": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(
        """
        Check if the document recommends a book. Say "yes" or "no".

        Recommended Books: 
        {recommended_books}

        Classification:"""
    )
    | llm
    | StrOutputParser()
)

# If yes
recommended_chain = (
    {"recommended_books" : RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are an expert in recommended books. \
        Give the user book recommendation books using below information. \
        Always start with "I have some books recommendation that is tailored to your taste. \
        
        Recommended Books: 
        {recommended_books}
        """
    )
    | llm
)

# If no
popular_chain = (
    {"recommended_books": popular_qa, "question": RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are an expert in recommended books. \
        Give the user book recommendation books using below information. \
        Always start with "I have some popular books that I can recommend for you. \
        
        Recommended Books: 
        {recommended_books}
        """
    )
    | llm
)

from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "yes" in x["topic"].lower() or "Yes" in x["topic"].lower(), recommended_qa | recommended_chain),
    (lambda x: "no" in x["topic"].lower() or "No" in x["topic"].lower(), popular_chain),
    popular_chain
)

full_chain = (
    {
        "topic": (
            {"recommended_books": recommended_qa, "query": RunnablePassthrough()}
            | ChatPromptTemplate.from_template(
                """
                Check if the document recommends a book. Say "yes" or "no".

                Recommended Books: 
                {recommended_books}

                Classification:"""
            )
            | llm
            | StrOutputParser()
            ), 
        "query": RunnablePassthrough()
    }
    | branch
    | StrOutputParser()
)

In [120]:
async for chunk in full_chain.astream_log(
    "I am user 1 and recommend me books about history"
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '7fd58734-c15c-41dc-88ca-4c716e0e15bb',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel',
  'value': {'end_time': None,
            'final_output': None,
            'id': '6a21ef2a-9a10-42fc-8977-5be9e85078c0',
            'metadata': {},
            'name': 'RunnableParallel',
            'start_time': '2023-11-01T06:50:14.261',
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'dd014cbc-c3c3-4d3e-9fa2-67d3c3ad1bed',
            'metadata': {},
            'name': 'RunnableSequence',
            'star

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:371: UserWarning: The aapply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI/final_output',
  'value': {'generations': [[{'generation_info': {'finish_reason': 'stop'},
                              'message': AIMessage(content='This document does not answer the question\nScore: 0'),
                              'text': 'This document does not answer the '
                                      'question\n'
                                      'Score: 0',
                              'type': 'ChatGeneration'}]],
            'llm_output': {'model_name': 'gpt-4',
                           'token_usage': {'completion_tokens': 12,
                                           'prompt_tokens': 335,
                                           'total_tokens': 347}},
            'run': None}},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/end_time',
  'value': '2023-11-01T06:50:19.762'})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:371: UserWarning: The aapply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:7/final_output',
  'value': {'generations': [[{'generation_info': {'finish_reason': 'stop'},
                              'message': AIMessage(content='This document does not answer the question\nScore: 0'),
                              'text': 'This document does not answer the '
                                      'question\n'
                                      'Score: 0',
                              'type': 'ChatGeneration'}]],
            'llm_output': {'model_name': 'gpt-4',
                           'token_usage': {'completion_tokens': 12,
                                           'prompt_tokens': 335,
                                           'total_tokens': 347}},
            'run': None}},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:7/end_time',
  'value': '2023-11-01T06:50:26.764'})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOp

In [121]:
async for chunk in full_chain.astream_log(
    "I am user 1 and recommend me books about romance"
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '668f59cd-16a3-40bb-ae9e-eb10f444c6d6',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'ae0b0a0c-00c9-4984-9558-bc063a6c87b5',
            'metadata': {},
            'name': 'RunnableParallel',
            'start_time': '2023-11-01T06:55:34.136',
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/<lambda>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '8857e759-2956-4e95-9d63-3b00a4d7aabf',
            'metadata': {},
            'name': '<lambda>',
            'start_time': '2023-1

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:371: UserWarning: The aapply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI/final_output',
  'value': {'generations': [[{'generation_info': {'finish_reason': 'stop'},
                              'message': AIMessage(content='This document does not answer the question\nScore: 0'),
                              'text': 'This document does not answer the '
                                      'question\n'
                                      'Score: 0',
                              'type': 'ChatGeneration'}]],
            'llm_output': {'model_name': 'gpt-4',
                           'token_usage': {'completion_tokens': 12,
                                           'prompt_tokens': 335,
                                           'total_tokens': 347}},
            'run': None}},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/end_time',
  'value': '2023-11-01T06:55:37.579'})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:371: UserWarning: The aapply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:7/final_output',
  'value': {'generations': [[{'generation_info': {'finish_reason': 'stop'},
                              'message': AIMessage(content='This document does not answer the question\nScore: 0'),
                              'text': 'This document does not answer the '
                                      'question\n'
                                      'Score: 0',
                              'type': 'ChatGeneration'}]],
            'llm_output': {'model_name': 'gpt-4',
                           'token_usage': {'completion_tokens': 12,
                                           'prompt_tokens': 350,
                                           'total_tokens': 362}},
            'run': None}},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:7/end_time',
  'value': '2023-11-01T06:55:42.564'})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOp

In [126]:
full_chain.invoke("I am user 1 and recommend me books about history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'I have some book recommendations that are tailored to your taste. \n\n1. "Informal Economy: A Research G" - This book, first published in 1991, is an imprint of Taylor & Francis, an informa company. \n\n2. "Hooded Americanism: The History of the Ku Klux Klan, 3rd ed." - This is the only work that treats Ku Kluxism for the entire period of its existence. It is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. \n\n3. "Environmental Regulation: Statutory Supplement and Guide to Internet Resources, 2002 Edition" - This excellent resource takes full advantage of online information and students\' familiarity with it. It reflects the author\'s dual focus on environmental regulations and the policy considerations that shape them.\n\n4. "Vision & Prophecy in Amos Revised" - This book presents the faculty lectures at the Baptist Theological Seminary in Ruschlikon in 1955. It explores Amos forty years later both in terms of its literary w

### Tool Consolidation

In [129]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Generic Recommendation',
        func=qa.run,
        description=(
            'use this tool when the user asking for book recommendation without any specific preference'
        )
    ),
    Tool(
        name='Specific Recommendation',
        func=full_chain.invoke,
        description=(
            'use this tool when the user asking for book recommendation with a specific preferenc'
        )
    ),
    Tool(
        name='Popular Recommendation',
        func=popular_qa.run,
        description=(
            'use this tool when the user asking for popular book recommendation without any specific preference'
        )
    ),
]

## Conversation Agent

In [130]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [131]:
agent("Recommend me some books about history")



> Entering new AgentExecutor chain...
```json
{
    "action": "Specific Recommendation",
    "action_input": "history"
}
```

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



Observation: I have some book recommendations that are tailored to your taste. 

1. "Vision & Prophecy in Amos Revised" by John D. W. Watts: This book provides a form-critical analysis of the visions of Amos and has significantly impacted prophetic studies. It's a must-read for anyone interested in biblical prophecy.

2. "Hooded Americanism: The History of the Ku Klux Klan, 3rd ed.": This is the only work that treats Ku Kluxism for the entire period of its existence. It provides a comprehensive view of a century of terror in American life.

3. "Environmental Regulation: Statutory Supplement and Guide to Internet Resources, 2002 Edition": This resource takes full advantage of online information and students' familiarity with it. It reflects the author's dual focus on environmental regulations and the policy considerations that shape them.

4. "Informal Economy: A Research G": Published in 1991, this book provides valuable insights into the informal economy.

5. "In Amma's Healing Room:

{'input': 'Recommend me some books about history',
 'chat_history': [],
 'output': "Based on your interest in history, here are some book recommendations: \n\n1. 'Vision & Prophecy in Amos Revised' by John D. W. Watts: This book provides a form-critical analysis of the visions of Amos and has significantly impacted prophetic studies. It's a must-read for anyone interested in biblical prophecy.\n\n2. 'Hooded Americanism: The History of the Ku Klux Klan, 3rd ed.': This is the only work that treats Ku Kluxism for the entire period of its existence. It provides a comprehensive view of a century of terror in American life.\n\n3. 'Environmental Regulation: Statutory Supplement and Guide to Internet Resources, 2002 Edition': This resource takes full advantage of online information and students' familiarity with it. It reflects the author's dual focus on environmental regulations and the policy considerations that shape them.\n\n4. 'Informal Economy: A Research G': Published in 1991, this book